In [8]:
import tensorflow as tf
import pandas as pd
tf.__version__

'2.14.0'

In [9]:
data = pd.read_csv("../input_data/CSVs/connect-4.csv")
data.head()

,a1,a2,a3,a4,a5,a6,b1,b2,b3,b4,...,f4,f5,f6,g1,g2,g3,g4,g5,g6,class
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
2,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,2
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [10]:
target = data.pop('class')
target.head()

0    2
1    2
2    2
3    2
4    2
Name: class, dtype: int64

**Non regulated model**

In [11]:
tf.random.set_seed(8)
model = tf.keras.Sequential()

fc1 = tf.keras.layers.Dense(512, input_shape=(42,), activation='relu')
fc2 = tf.keras.layers.Dense(512, activation='relu')
fc3 = tf.keras.layers.Dense(128, activation='relu')
fc4 = tf.keras.layers.Dense(128, activation='relu')
out = tf.keras.layers.Dense(3, activation='softmax')

model.add(fc1)
model.add(fc2)
model.add(fc3)
model.add(fc4)
model.add(out)
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 512)               22016     
                                                                 
 dense_11 (Dense)            (None, 512)               262656    
                                                                 
 dense_12 (Dense)            (None, 128)               65664     
                                                                 
 dense_13 (Dense)            (None, 128)               16512     
                                                                 
 dense_14 (Dense)            (None, 3)                 387       
                                                                 
Total params: 367235 (1.40 MB)
Trainable params: 367235 (1.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


### *NOTE: Here, the `legacy` Adam optimiser is used as the non-legacy variant throws an error - this seems to be due to the fact that GPU is unsupported on windows after 2.10. Using WSL2 should solve the issue, as stated [here](https://stackoverflow.com/questions/74684240/keyerror-the-optimizer-cannot-recognize-variable-dense-1-kernel0-for-pretrai)*

In [12]:
# Only 1 output should be confidently selected
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam(0.001)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
model.fit(data, target, epochs=5, validation_split=0.2)

Epoch 1/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.6399 - accuracy: 0.7465 - val_loss: 1.0103 - val_accuracy: 0.5808
Epoch 2/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.4955 - accuracy: 0.8054 - val_loss: 1.2081 - val_accuracy: 0.5682
Epoch 3/5
1689/1689 [==============================] - 8s 5ms/step - loss: 0.4374 - accuracy: 0.8272 - val_loss: 1.3383 - val_accuracy: 0.5650
Epoch 4/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.4040 - accuracy: 0.8412 - val_loss: 1.4464 - val_accuracy: 0.5725
Epoch 5/5
1689/1689 [==============================] - 8s 5ms/step - loss: 0.3757 - accuracy: 0.8508 - val_loss: 1.3776 - val_accuracy: 0.5847


**Model using L2 regulariser**

In [13]:
reg_model = tf.keras.models.Sequential()

reg_fc1 = tf.keras.layers.Dense(512, input_shape=(42,), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
reg_fc2 = tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
reg_fc3 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
reg_fc4 = tf.keras.layers.Dense(128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l=0.1))
reg_fc5 = tf.keras.layers.Dense(3, activation='softmax')

reg_model.add(reg_fc1)
reg_model.add(reg_fc2)
reg_model.add(reg_fc3)
reg_model.add(reg_fc4)
reg_model.add(reg_fc5)
reg_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 512)               22016     
                                                                 
 dense_16 (Dense)            (None, 512)               262656    
                                                                 
 dense_17 (Dense)            (None, 128)               65664     
                                                                 
 dense_18 (Dense)            (None, 128)               16512     
                                                                 
 dense_19 (Dense)            (None, 3)                 387       
                                                                 
Total params: 367235 (1.40 MB)
Trainable params: 367235 (1.40 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Optimiser and loss functions used are identical to that of the un-regularised model
reg_model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])
reg_model.fit(data, target, epochs=5, validation_split=0.2)

Epoch 1/5
1689/1689 [==============================] - 9s 5ms/step - loss: 1.1639 - accuracy: 0.6780 - val_loss: 0.9637 - val_accuracy: 0.5796
Epoch 2/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.8169 - accuracy: 0.6780 - val_loss: 0.9700 - val_accuracy: 0.5796
Epoch 3/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.8172 - accuracy: 0.6780 - val_loss: 0.9506 - val_accuracy: 0.5796
Epoch 4/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.8170 - accuracy: 0.6780 - val_loss: 0.9609 - val_accuracy: 0.5796
Epoch 5/5
1689/1689 [==============================] - 9s 5ms/step - loss: 0.8169 - accuracy: 0.6780 - val_loss: 0.9693 - val_accuracy: 0.5796


*While performance here is not great, the much slimmer difference between `accuracy` and `val_accuracy` shows a big improvement in its predictions outside the training dataset in comparison to within it*